# Workshop 8: Polynomial Regression On Anscombe's dataset

## Importing the libraries

In [2]:
import numpy as np
import pymc3 as pm
import pandas as pd
import matplotlib.pyplot as plt

## Extracting the data and visualising it

In [ ]:
ansc = sns.load_dataset('anscombe') # loading the dataset
x_2 = ans[ans.group == 'II']['x'].values
y_2 = ans[ans.group == 'II']['y'].values
x_2 = x_2 - x_2.mean()
plt.scatter(x_2, y_2)
plt.xlabel('x')
plt.ylabel('y', rotation=0)